# Naver 영화 Review Crawling Sample Code

## [Keras]영화 평점, 줄거리를 가지고 평점 예측 모델 만들어보기

데이터 수집

https://somjang.tistory.com/entry/Python%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%98%81%ED%99%94-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0

데이터 가공
https://somjang.tistory.com/entry/Python%EC%98%81%ED%99%94-%ED%8F%89%EC%A0%90-%EC%A4%84%EA%B1%B0%EB%A6%AC%EB%A5%BC-%EA%B0%80%EC%A7%80%EA%B3%A0-%ED%8F%89%EC%A0%90-%EC%98%88%EC%B8%A1-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EC%96%B4%EB%B3%B4%EA%B8%B0feat-Keras

In [3]:
import requests 
from bs4 import BeautifulSoup 
from selenium import webdriver as wd 
from selenium.webdriver.common.keys import Keys 

import pandas as pd

import time 
import re 

In [4]:
def getPageLinks(pageRange): 
    links = [] 
    
    for pageNo in range(pageRange): 
        url = "https://series.naver.com/movie/recentList.nhn?orderType=sale&sortingType=&tagCode=&page=" + str(pageNo + 1) 
        req = requests.get(url) 
        soup = BeautifulSoup(req.text, 'lxml') 
        movielinks = soup.select('div.lst_thum_wrap ul li a[href]') 
        
        for movielink in movielinks: 
            link = str(movielink.get('href')) 
            links.append("https://series.naver.com"+link) 
    return links

In [5]:
def getPageLinksWantRange(startPageNo, lastPageNo): 
    links = [] 
    return_links = [] 
    
    for pageNo in range(startPageNo-1, lastPageNo): 
        url = "https://series.naver.com/movie/recentList.nhn?orderType=sale&sortingType=&tagCode=&page=" + str(pageNo + 1) 
        req = requests.get(url) 
        soup = BeautifulSoup(req.text, 'lxml') 
        movielinks = soup.select('div.lst_thum_wrap ul li a[href]') 
        
        for movielink in movielinks: 
            link = str(movielink.get('href')) 
            links.append("https://series.naver.com"+link) 
    return links

In [6]:
def getMovieDataFromNaverSeries(links): 
    title_infos = [] 
    content_infos = [] 
    genre_infos = [] 
    score_infos = [] 
    date_infos = [] 
    url2 = "https://www.naver.com" 
    
    driver = wd.Chrome(executable_path="D:/Anaconda3/chromedriver.exe") 
    driver.get(url2) 
    time.sleep(3.0) # 30 
    
    driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t") 
    
    for link in links: 
        driver.switch_to.window(driver.window_handles[-1]) 
        time.sleep(0.1) 
        driver.get(link) 
        time.sleep(0.1) 
        driver.switch_to.window(driver.window_handles[0]) 
        time.sleep(0.3) 
        
        html_source = driver.page_source 
        
        html_soup = BeautifulSoup(html_source, 'lxml') 
        
        flag = html_soup.text[0:10] 
        
        newflag = "".join(flag) 
        newflag = newflag.replace('\n', '') 
        
        if newflag == '네이버': 
            time.sleep(1.0) 
            score = driver.find_element_by_css_selector('div.score_area > em ') 
            
            score = float(score.text) 
            score = int(score) 
            score_infos.append(score) 
            
            genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent') 
            genre = genre.replace('장르','') 
            genre = genre.split('/') 
            genre_infos.append(genre) 
            text = driver.find_element_by_css_selector('span.al_r > a') #.click() 
            
            # review_url = text.get_attribute('href') 
            # review_url = review_url.replace('basic', 'pointWriteFormList') 
            
            # review_url = review_url + '&type=after&onlyActualPointYn=N&order=newest&page=1' 
            
            movieInfoUrl = text.get_attribute('href') 
            
            movie_req = requests.get(movieInfoUrl) 
            
            movie_soup = BeautifulSoup(movie_req.text, 'lxml') 
            
            titles = movie_soup.select('div.mv_info > h3.h_movie > a') 
            
            temp_titles = [] 
            
            for title in titles: 
                temp = title.text 
                temp = temp.replace('상영중', '') 
                temp = temp.replace('\n', '') 
                temp_titles.append(temp) 
                
                if '' in temp_titles or ' ' in temp_titles: 
                    temp_titles.remove('') 
                    
                temp_titles = set(temp_titles) 
                temp_titles = list(temp_titles) 
                temp_titles = [x for x in temp_titles if x is not ''] 
                title_infos.append(list(temp_titles)[0]) 
                
                contents_texts = movie_soup.select('div.story_area > p.con_tx') 
                
                if len(contents_texts) == 0: 
                    content_infos.append("줄거리 오류") 
                else: 
                    for contents in contents_texts: 
                        temp = contents.text 
                        temp = temp.replace('\r', '') 
                        temp = temp.replace('\xa0', '') 
                        content_infos.append(temp) 
                        
        elif newflag == '네이버 :': 
            adult_movies.append(link) 
            
    print(len(score_infos), len(genre_infos), len(content_infos)) 
    
    driver.close() 
    
    movie_dic = {"평점":score_infos,"장르":genre_infos, "줄거리":content_infos} 
    
    movie_df = pd.DataFrame(movie_dic, index=title_infos) 
    
    movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') 
    
    return movie_df2

In [7]:
def dftoCsv(movie_df, num):
    try: 
        movie_df.to_csv(('movie_data'+ str(num) +'.csv'), sep=',', na_rep='NaN', encoding='euc-kr') 
    except: 
        print("Error")

## 크롤링과 코사인 유사도를 이용하여 영화추천 서비스를 만들어보자

https://steadyman.tistory.com/10

In [8]:
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm_notebook
import nltk
import re
from urllib.request import urlopen

In [30]:
domain='https://movie.naver.com'
story=[]
title=[]
genre=[]
for i in tqdm_notebook(range (1,11)):
    
    url="https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20191201&page="+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    titles=soup.find_all('div',class_='tit5')
    hype=[]
    href=[]
  
    try:
        
        for each in titles:
           
                hype=each.find_all('a')
                for link in hype:
                    href.append(link['href'])
        for j in tqdm_notebook(range(len(href))):
                domain='https://movie.naver.com'
                domain=domain+href[j]
                html=urlopen(domain)
                soup=BeautifulSoup(html,"html.parser")
                story.append(soup.find('p',class_="con_tx").get_text())
            
                title_tag=soup.find('h3',class_='h_movie')
                title.append(title_tag.find('a').get_text())
            
                genre_tag=soup.find('p')
                genre.append(genre_tag.find('a').get_text())
    except:
        pass
                  

#스토리 정규화 처리
import re

for i in tqdm_notebook(range(len(story))):

    story[i] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》“”’]','',story[i] )
    story[i] = re.sub('\r\xa0','',story[i] )

d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [31]:
title

['그린 북',
 '가버나움',
 '베일리 어게인',
 '아일라',
 '주전장',
 '원더',
 '당갈',
 '쇼생크 탈출',
 '터미네이터 2:오리지널',
 '보헤미안 랩소디',
 '덕구',
 '월-E',
 '나 홀로 집에',
 '인생은 아름다워',
 '빽 투 더 퓨쳐',
 '살인의 추억',
 '매트릭스',
 '헬프',
 '라이언 일병 구하기',
 '포레스트 검프',
 '사운드 오브 뮤직',
 '글래디에이터',
 '알라딘',
 '어벤져스: 엔드게임',
 '위대한 쇼맨',
 '센과 치히로의 행방불명',
 '안녕 베일리',
 '토이 스토리 3',
 '죽은 시인의 사회',
 '헌터 킬러',
 '클래식',
 '레옹',
 '아이 캔 스피크',
 '반지의 제왕: 왕의 귀환',
 '동주',
 '굿바이 마이 프랜드',
 '캐스트 어웨이',
 '여인의 향기',
 '히든 피겨스',
 '주토피아',
 '집으로...',
 '타이타닉',
 '서유기 2 - 선리기연',
 '클레멘타인',
 '세 얼간이',
 '쉰들러 리스트',
 '굿 윌 헌팅',
 '지금, 만나러 갑니다',
 '울지마 톤즈',
 '모노노케 히메',
 '에이리언 2',
 '언터처블: 1%의 우정',
 '다시 태어나도 우리',
 '아마데우스',
 '달링',
 '그대를 사랑합니다',
 '다크 나이트',
 '반지의 제왕: 두 개의 탑',
 '아이언 자이언트',
 '패왕별희',
 '미세스 다웃파이어',
 '더 록',
 '트루먼 쇼',
 '드래곤 길들이기',
 '아이 엠 샘',
 '라푼젤',
 '폴란드로 간 아이들',
 '프리퀀시',
 '소원',
 '모던 타임즈',
 '해리 포터와 죽음의 성물 - 2부',
 '빌리 엘리어트',
 '나 홀로 집에 2 - 뉴욕을 헤매다',
 '번 더 스테이지: 더 무비',
 '주먹왕 랄프',
 '미스 슬로운',
 '피아니스트',
 '지상의 별처럼',
 '코러스',
 '러브 유어셀프 인 서울',
 '로망',
 '바이센테니얼 맨',
 '업',
 '블라인드 사이드',
 '언더독',
 '알라딘',
 '아기

In [32]:
genre

['드라마',
 '드라마',
 '모험',
 '드라마',
 '다큐멘터리',
 '드라마',
 '드라마',
 '드라마',
 'SF',
 '드라마',
 '드라마',
 '애니메이션',
 '모험',
 '드라마',
 'SF',
 '범죄',
 'SF',
 '드라마',
 '전쟁',
 '드라마',
 '멜로/로맨스',
 '액션',
 '모험',
 '액션',
 '드라마',
 '애니메이션',
 '가족',
 '애니메이션',
 '드라마',
 '액션',
 '멜로/로맨스',
 '범죄',
 '드라마',
 '판타지',
 '드라마',
 '드라마',
 '드라마',
 '드라마',
 '드라마',
 '애니메이션',
 '드라마',
 '멜로/로맨스',
 '코미디',
 '액션',
 '코미디',
 '드라마',
 '드라마',
 '멜로/로맨스',
 '다큐멘터리',
 '애니메이션',
 'SF',
 '코미디',
 '다큐멘터리',
 '드라마',
 '드라마',
 '드라마',
 '액션',
 '판타지',
 '애니메이션',
 '드라마',
 '코미디',
 '액션',
 '코미디',
 '애니메이션',
 '드라마',
 '애니메이션',
 '다큐멘터리',
 '범죄',
 '드라마',
 '코미디',
 '모험',
 '드라마',
 '모험',
 '한국',
 '애니메이션',
 '드라마',
 '드라마',
 '드라마',
 '드라마',
 '공연실황',
 '멜로/로맨스',
 'SF',
 '애니메이션',
 '드라마',
 '애니메이션',
 '애니메이션',
 '애니메이션',
 '애니메이션',
 '드라마',
 'SF',
 '코미디',
 '애니메이션',
 '드라마',
 '드라마',
 '애니메이션',
 '액션',
 '가족',
 'SF',
 '드라마',
 '다큐멘터리',
 '다큐멘터리',
 '애니메이션',
 '드라마',
 '드라마',
 '애니메이션',
 '판타지',
 '액션',
 '액션',
 '드라마',
 '판타지',
 '애니메이션',
 '애니메이션',
 '공연실황',
 '애니메이션',
 '드라마',
 '드라마',
 '코미디',
 '다큐멘터리',
 '드라마',
 '액

In [33]:
story

['1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아함 그 자체인천재 피아니스트 돈 셜리마허샬라 알리 박사의 운전기사 면접을 보게 된다백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사생각 행동 말투 취향까지 달라도 너무 다른 두 사람은그들을 위한 여행안내서 그린북에 의존해 특별한 남부 투어를 시작하는데',
 '나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 12살 소년 자인으로부터',
 '귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다시 시작된 견생 2회차 아니 3회차1등 경찰견 엘리에서 찰떡같이 마음을 알아주는 소울메이트 티노까지다시 태어날 때마다 성별과 생김새 직업에 이름도 바뀌지만여전히 영혼만은 사랑 충만 애교 충만 주인바라기 베일리어느덧 견생 4회차 방랑견이 되어 떠돌던 베일리는마침내 자신이 돌아온 진짜 이유를 깨닫고 어딘가로 달려가기 시작하는데',
 '1950년 한국전쟁에 파병된 슐레이만은 칠흑 같은 어둠 속 홀로 남겨진 5살 소녀를 발견한다\xa0전쟁과 부모를 잃은 충격 속에 말을 잃은 소녀 슐레이만은 소녀에게 터키어로 달이라는 뜻의 아일라라는 이름을 지어주고 함께 부대로 향한다 서로에게 무엇과도 바꿀 수 없는 가장 소중한 존재가 된 두 사람 그러나 행복도 잠시 슐레이만은 종전과 함께 고국으로 돌아가라는 명령을 받게 되고 아일라를 끝까지 지켜내기 위해 모두의 반대 속에 자신의 선택을 감행하게 되는데\xa0\xa0전 세계를 감동시킨 위대한 이야기가 찾아온다',
 '일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인 유튜버 미키 데자키 그는 일본군위안부에 관한 기사를 쓴 기자가 우익들에게 인

In [14]:
Nmovie=pd.DataFrame(data={'제목':title,'줄거리':story,'장르':genre})

In [16]:
Nmovie.head()

,제목,줄거리,장르
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험
3,아일라,1950년 한국전쟁에 파병된 슐레이만은 칠흑 같은 어둠 속 홀로 남겨진 5살 소녀를...,드라마
4,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리


In [17]:
Nmovie['합침'] = (Nmovie['제목']) + Nmovie['줄거리'] + (Nmovie['장르'])

In [18]:
Nmovie['합침'][0]

'그린 북1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아함 그 자체인천재 피아니스트 돈 셜리마허샬라 알리 박사의 운전기사 면접을 보게 된다백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사생각 행동 말투 취향까지 달라도 너무 다른 두 사람은그들을 위한 여행안내서 그린북에 의존해 특별한 남부 투어를 시작하는데드라마'

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
Nmovie['합침'] = Nmovie['합침'].fillna('')

In [21]:
tfidf_matrix = tfidf.fit_transform(movie)
# overview에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(4, 4)


In [22]:
#코사인유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [23]:
indices = pd.Series(Nmovie.index, index=Nmovie['제목']).drop_duplicates()
print(indices.head())
#영화의 타이틀과 인덱스를 가진 테이블을 만듬
#영화 타이틀을 입력하면 인덱스를 리턴하려고 만듬

제목
그린 북       0
가버나움       1
베일리 어게인    2
아일라        3
주전장        4
dtype: int64


In [24]:
def get_recomm(title, cosine_sim=cosine_sim):
    choice = []
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]
    
    for i in range(10):
        choice.append(Nmovie['제목'][movie_indices[i]])
    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    print('***영화 추천 순위***')
    for i in range(10):
        print(str(i+1) + '순위 : ' + choice[i])

In [25]:
get_recomm('토이 스토리')

IndexError: index 91 is out of bounds for axis 0 with size 4